In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def simulate_path(s0, mu, sigma, horizon, timesteps, n_sims, seed=None):
    if seed is not None:
        np.random.seed(seed)
    dt = horizon / timesteps
    S = np.zeros((timesteps, n_sims))
    S[0, :] = s0
    for i in range(1, timesteps):
        w = np.random.standard_normal(n_sims)
        S[i, :] = S[i-1, :] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * w)
    return S



In [ ]:
def simulate_path_milstein(s0, mu, sigma, horizon, timesteps, n_sims, seed=None):
    if seed is not None:
        np.random.seed(seed)
    
    dt = horizon / timesteps  # Time step
    S = np.zeros((timesteps, n_sims))  # Initialize stock price matrix
    S[0, :] = s0  # Set initial stock price
    
    for i in range(1, timesteps):
        w = np.random.standard_normal(n_sims)  # Standard normal random variable
        dW = np.sqrt(dt) * w  # Brownian motion increment
        
        # Milstein correction term
        correction = 0.5 * sigma**2 * S[i-1, :] * (dW**2 - dt)
        
        # Apply the Milstein update
        S[i, :] = S[i-1, :] + mu * S[i-1, :] * dt + sigma * S[i-1, :] * dW + correction
    
    return S


In [ ]:
price_path = pd.DataFrame(simulate_path(242.95,0.05,0.2,1,252,100000))
# Plot the histogram of the simulated price path at maturity
price_path.iloc[-1].hist(bins=100, color='orange', label='End-Period Price in Histogram')
plt.legend()
plt.title('Simulated Geometric Brownian Motion at Maturity')

price_path = pd.DataFrame(simulate_path(242.95,0.05,0.2,1,252,100000))
# Plot the histogram of the simulated price path at maturity
price_path.iloc[-1].hist(bins=100, color='red', label='End-Period Price in Histogram')
plt.legend()
plt.title('Simulated Geometric Brownian Motion at Maturity')

In [ ]:
def simulate_acdc(S0, r, sigma, T, N_paths, N_periods, days_per_year, m, K, B, G, contract='accumulator', seed=None):
    n_steps = int(days_per_year * T)
    #S = simulate_path(S0, r, sigma, T, n_steps, N_paths, seed)
    S = simulate_path_milstein(S0, r, sigma, T, n_steps, N_paths, seed)
    delta = 1 if contract.lower() == 'accumulator' else -1
    period_indices = np.linspace(0, n_steps, N_periods + 1, dtype=int)
    period_payoffs = np.zeros((N_paths, N_periods))
    for i in range(N_periods):
        start_index = period_indices[i]
        settlement_index = period_indices[i + 1] - 1
        X_i = np.zeros(N_paths)
        days_in_period = settlement_index - start_index + 1
        for day in range(days_in_period):
            for sim in range(N_paths):
                price = S[start_index + day, sim]
                if price < B:
                    if delta * (price - K) >= 0:
                        X_i[sim] += m
                    else:
                        X_i[sim] += G * m
                else:
                    X_i[sim] += 0
        S_settlement = S[settlement_index, :]
        A_i = delta * (S_settlement - K)
        period_payoff = X_i * A_i
        t_settlement = settlement_index / days_per_year
        discount_factor = np.exp(-r * t_settlement)
        #discount_factor = 0.995
        period_payoffs[:, i] = period_payoff * discount_factor
        
        # Print discount factor and the average discounted payoff for the period
        avg_payoff = np.mean(period_payoffs[:, i])
        print(f"Period {i+1}: t_settlement = {t_settlement*days_per_year:.3f} days, discount_factor = {discount_factor:.9f}, avg discounted payoff = {avg_payoff:.4f}")

    
    total_payoffs = np.sum(period_payoffs, axis=1)
    price = np.mean(total_payoffs)
    return price


In [ ]:
if __name__ == '__main__':
    S0 = 214.945
    r = 0.0584
    sigma = 0.0
    T = 12/12
    days_per_year = 252
    N_periods = 12
    N_paths = 10000
    m = 1.0
    K = 214.945
    B = 1000.0
    G = 1
    contract_type = 'accumulator'
    
    price = simulate_acdc(S0, r, sigma, T, N_paths, N_periods, days_per_year, m, K, B, G, contract=contract_type, seed=100)
    print(f"Estimated price of the {contract_type} ACDC contract: {price:.4f}")


In [ ]:
import random

# Number of random points to generate
N = 1000000
inside_circle = 0

for _ in range(N):
    x = random.uniform(-1, 1)
    y = random.uniform(-1, 1)
    
    if x**2 + y**2 <= 1:
        inside_circle += 1

# Estimate of π
pi_estimate = 4 * inside_circle / N
print(f"Estimated value of π = {pi_estimate}")


In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np

def estimate_pi(n):
    inside = 0
    for _ in range(n):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        if x**2 + y**2 <= 1:
            inside += 1
    pi_est = 4 * inside / n
    return pi_est

# Sample sizes to test
sample_sizes = [10, 100, 1000, 5000, 10000, 50000, 100000, 200000, 500000]
true_pi = np.pi
errors = []

# Run simulation and calculate absolute error for each sample size
for n in sample_sizes:
    pi_hat = estimate_pi(n)
    error = abs(pi_hat - true_pi)
    errors.append(error)
    print(f"n = {n:<7} | π estimate = {pi_hat:.6f} | Error = {error:.6f}")

# Plotting the error vs. sample size on log-log scale
plt.figure(figsize=(8, 5))
plt.loglog(sample_sizes, errors, 'o-', label='Empirical Error')
plt.loglog(sample_sizes, [1/np.sqrt(n) for n in sample_sizes], '--', label='1/√n Reference')
plt.xlabel('Number of samples (n)')
plt.ylabel('Absolute error')
plt.title('Monte Carlo Error in Estimating π')
plt.grid(True, which='both')
plt.legend()
plt.tight_layout()
plt.show()


In [4]:
n = 100000
n_c = 0
import numpy as np
for _ in range (n):
    x = np.random.uniform(-1,1)
    y = np.random.uniform(-1,1)
    d = x**2 + y**2
    if d <= 1:
        n_c += 1
pi = 4*(n_c/n)
print(pi)

3.14024


1. Why was the option price turning negative?

Was this due to numerical integration error, extrapolation, poor tail behavior, or a deeper modeling issue?

2. In what scenarios or regions of the input space did negative prices occur?

E.g., low vol, short maturity, deep out-of-the-money?

3. How frequently did this occur in production or backtesting?

Was this an edge case or a systemic issue?

5. Why is a zero floor the best fix?

Were alternative fixes considered? (e.g., refining integration scheme, regularization, constrained optimization)

6. What is the impact of the zero floor on option sensitivities (Greeks)?

Are the first and second derivatives (e.g., delta, gamma) now discontinuous near zero price?

7. Confirm whether the fix changes any P&L impact in production.

8. Have you tested a broader set of trades (different maturities, volatilities, moneyness) to ensure robustness of the fix?

9. How does the fix affect sensitivities, particularly delta and gamma, near the zero threshold?

10. Is there logging or a flag in the code that tracks when the zero floor is applied?

11. Have you checked for any P&L jumps across this change for impacted trades?

